# Import Libraries

In [202]:
import os
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import classification_report

# Set Parameters

In [203]:
dataset_nr = 2
assert dataset_nr == 1 or 2

embedding_method = 'e2v'
assert embedding_method == 'e2v' or 'm2v'

# Additional experiments
NO_GENE_PRODUCTS = 'no gene products'
SINGLE_RELATION_TYPE = 'only one relation type'
RANDOM_FEATURES = 'random node embedding values'
concept_changes = None  # set to None when no conceptual changes are wanted

if concept_changes:
    concept_changes_str = concept_changes
else:
    concept_changes_str = ''
    
if concept_changes == NO_GENE_PRODUCTS:
    suffix = '_nogeneprods'
elif concept_changes == SINGLE_RELATION_TYPE:
    suffix = '_singlerel'
elif concept_changes == RANDOM_FEATURES:
    suffix = '_randomfeat'
else:
    suffix = ''

seeded_emb = False

if seeded_emb:
    fixed_emb = '_seeded'
else:
    fixed_emb = ''
    
if seeded_emb:
    title_seeded = ' with fixed node embeddings'
else:
    title_seeded = ''

# Get Result Paths

In [204]:
curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')
dataset_output_dir = os.path.join(curr_output_dir, f'g{dataset_nr}_{embedding_method}{fixed_emb}{suffix}')

if not os.path.exists(dataset_output_dir):
    print('First, run the edge2vec embedding script. Then, run this script.')
else:
    print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {dataset_output_dir}')
    
run_folders_list = []
for item in os.listdir(dataset_output_dir):
    curr_path = os.path.join(dataset_output_dir, item)
    if os.path.isdir(curr_path) and 'run' in item:
        run_folders_list.append(item)

print(f'A total of {len(run_folders_list)} runs will be included in the analysis.')

run_folders_paths = []
pred_folders_paths = []
for run_folder in run_folders_list:
    run_path = os.path.join(dataset_output_dir, run_folder)
    run_folders_paths.append(run_path)
    pred_run_path = os.path.join(run_path, 'pred')
    pred_folders_paths.append(pred_run_path)
    print(pred_run_path)

Output folder for dataset 2 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat
A total of 10 runs will be included in the analysis.
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_001\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_002\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_003\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_004\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_005\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_006\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat\run_007\pred
c:\Users\

In [205]:
auc_scores_all_runs = []
auc_loss_scores_all_runs = []

for run_name, pred_folder in zip(run_folders_list, pred_folders_paths):
    with open(f'{pred_folder}/performance_scores_{dataset_nr}_{embedding_method}{suffix}.pkl', 'rb') as f:
        loaded_info = pickle.load(f)
        
    keys = ['AUC Train', 'AUC Validation', 'AUC Test']
    for key in keys:
        auc_scores = loaded_info[key]
        for index, auc_score in enumerate(auc_scores):
            auc_scores_per_run = {'run': run_name, 'name': key, 'iteration': index, 'score': auc_score}
            auc_scores_all_runs.append(auc_scores_per_run)
            auc_loss_scores_all_runs.append(auc_scores_per_run)

    loss_scores = loaded_info['Loss']
    for index, loss_score in enumerate(loss_scores):
        formatted_loss_score = float(np.log10(loss_score))
        loss_scores_per_run = {'run': run_name, 'name': 'Cross-Entropy Loss', 'iteration': index, 'score': formatted_loss_score}
        auc_loss_scores_all_runs.append(loss_scores_per_run)

# Plot ROC Curves, AUC-ROC Scores and F1 Scores for Each Model

## ROC Curves

In [206]:
all_settings = [
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 1},
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 2}
]

curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')

final_test_auc_roc_scores_all_runs_all_models = []
roc_curve_all_runs_all_models = []
f1_scores_all_runs_all_models = []

for setting in all_settings:
    curr_dataset_output_dir = os.path.join(curr_output_dir, f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}')

    if not os.path.exists(curr_dataset_output_dir):
        print('First, run the edge2vec embedding script. Then, run this script.')
    else:
        print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {curr_dataset_output_dir}')
            
        run_folders_list = []
        for item in os.listdir(curr_dataset_output_dir):
            curr_path = os.path.join(curr_dataset_output_dir, item)
            if os.path.isdir(curr_path) and 'run' in item:
                run_folders_list.append(item)

        print(f'A total of {len(run_folders_list)} runs will be included in the analysis.')

        run_folders_paths = []
        curr_pred_folders_paths = []
        for run_folder in run_folders_list:
            run_path = os.path.join(curr_dataset_output_dir, run_folder)
            run_folders_paths.append(run_path)
            pred_run_path = os.path.join(run_path, 'pred')
            curr_pred_folders_paths.append(pred_run_path)
            
        for run_name, pred_folder in zip(run_folders_list, curr_pred_folders_paths):
            with open(f'{pred_folder}/performance_scores_{setting["dataset_nr"]}_{setting["embedding_method"]}.pkl', 'rb') as f:
                loaded_info = pickle.load(f)

            auc_roc_score = loaded_info['ROC AUC Score']
            formatted_auc_roc_score = float(auc_roc_score)
            auc_roc_score_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}', 
                                        'ROC AUC Score': formatted_auc_roc_score}
            final_test_auc_roc_scores_all_runs_all_models.append(auc_roc_score_per_run)

            roc_fpr_scores = loaded_info['ROC FPR']
            roc_tpr_scores = loaded_info['ROC TPR']
            
            for fpr, tpr in zip(roc_fpr_scores, roc_tpr_scores):
                auc_per_threshold_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}',
                                             'ROC FPR': fpr, 'ROC TPR': tpr}
                roc_curve_all_runs_all_models.append(auc_per_threshold_per_run)

            f1_score = loaded_info['F1 Score']
            formatted_f1_score = float(f1_score)
            f1_score_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}', 
                                'F1 Score': formatted_f1_score}
            f1_scores_all_runs_all_models.append(f1_score_per_run)

            print(f'F1-Score in the test set of dataset {setting["dataset_nr"]} and method {setting["embedding_method"]}:', f1_score)
            print(classification_report(loaded_info['True Labels'], loaded_info['Predicted Labels']))

Output folder for dataset 2 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v
A total of 10 runs will be included in the analysis.
F1-Score in the test set of dataset 1 and method e2v: 0.9239595390267223
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     10412
           1       0.97      0.88      0.92     10412

    accuracy                           0.93     20824
   macro avg       0.93      0.93      0.93     20824
weighted avg       0.93      0.93      0.93     20824

F1-Score in the test set of dataset 1 and method e2v: 0.9186950813049186
              precision    recall  f1-score   support

           0       0.88      0.97      0.93     10412
           1       0.97      0.87      0.92     10412

    accuracy                           0.92     20824
   macro avg       0.93      0.92      0.92     20824
weighted avg       0.93      0.92      0.92     20824

F1-S

              precision    recall  f1-score   support

           0       0.92      0.90      0.91     11008
           1       0.91      0.93      0.92     11008

    accuracy                           0.91     22016
   macro avg       0.92      0.91      0.91     22016
weighted avg       0.92      0.91      0.91     22016

F1-Score in the test set of dataset 2 and method e2v: 0.9225507402907159
              precision    recall  f1-score   support

           0       0.93      0.91      0.92     11008
           1       0.91      0.93      0.92     11008

    accuracy                           0.92     22016
   macro avg       0.92      0.92      0.92     22016
weighted avg       0.92      0.92      0.92     22016

F1-Score in the test set of dataset 2 and method e2v: 0.919266875279392
              precision    recall  f1-score   support

           0       0.93      0.90      0.92     11008
           1       0.90      0.93      0.92     11008

    accuracy                         

In [207]:
roc_curve_all_runs_all_models = pd.DataFrame(roc_curve_all_runs_all_models)
roc_curve_all_runs_all_models

,Model,ROC FPR,ROC TPR
0,g1_e2v,0.000000,0.000000
1,g1_e2v,0.000000,0.000192
2,g1_e2v,0.000000,0.006531
3,g1_e2v,0.000000,0.006915
4,g1_e2v,0.000000,0.010949
...,...,...,...
36427,g2_e2v,0.875091,0.999637
36428,g2_e2v,0.875091,0.999818
36429,g2_e2v,0.934502,0.999818
36430,g2_e2v,0.934502,1.000000


In [208]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'ROC TPR/ROC FPR Distribution over Each Run for Each Model')
sns.scatterplot(data=roc_curve_all_runs_all_models, x="ROC FPR", y="ROC TPR", hue="Model", s=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(markerscale=10)

fig.savefig(f'{curr_output_dir}/roc_curves.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

## F1 Scores

In [209]:
f1_scores_all_runs_all_models = pd.DataFrame(f1_scores_all_runs_all_models)
f1_scores_all_runs_all_models

,Model,F1 Score
0,g1_e2v,0.923960
1,g1_e2v,0.918695
2,g1_e2v,0.921543
3,g1_e2v,0.926259
4,g1_e2v,0.924300
5,g1_e2v,0.925021
6,g1_e2v,0.923388
7,g1_e2v,0.926945
8,g1_e2v,0.920598
9,g1_e2v,0.926584


In [210]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'F1 Scores Overview for Each Model')
sns.barplot(f1_scores_all_runs_all_models, x="Model", y="F1 Score", errorbar="sd", color='cornflowerblue')
ax.bar_label(ax.containers[0], fontsize=10, padding=5)
ax.set_xlabel('Model Variant')
ax.set_ylabel('F1 Score')

fig.savefig(f'{curr_output_dir}/f1_scores.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

## Final AUC-ROC Scores

In [211]:
final_test_auc_roc_scores_all_runs_all_models = pd.DataFrame(final_test_auc_roc_scores_all_runs_all_models)
final_test_auc_roc_scores_all_runs_all_models

,Model,ROC AUC Score
0,g1_e2v,0.981466
1,g1_e2v,0.978083
2,g1_e2v,0.977793
3,g1_e2v,0.979621
4,g1_e2v,0.977190
5,g1_e2v,0.978637
6,g1_e2v,0.976998
7,g1_e2v,0.979838
8,g1_e2v,0.976336
9,g1_e2v,0.978401


In [212]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'AUC-ROC Scores Overview for Each Model')
sns.barplot(final_test_auc_roc_scores_all_runs_all_models, x="Model", y="ROC AUC Score", errorbar="sd")
ax.bar_label(ax.containers[0], fontsize=10, padding=10)
ax.set_ylim(0.85,1) # 0.85, 1
ax.set_xlabel('Model Variant')
ax.set_ylabel('AUC-ROC Score')

fig.savefig(f'{curr_output_dir}/auc_roc_scores.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

# Plot Training Curve

In [213]:
auc_scores_all_runs = pd.DataFrame(auc_scores_all_runs)
auc_scores_all_runs

,run,name,iteration,score
0,run_001,AUC Train,0,0.784185
1,run_001,AUC Train,1,0.796646
2,run_001,AUC Train,2,0.813906
3,run_001,AUC Train,3,0.831519
4,run_001,AUC Train,4,0.847915
...,...,...,...,...
2965,run_010,AUC Test,94,0.745730
2966,run_010,AUC Test,95,0.747070
2967,run_010,AUC Test,96,0.748329
2968,run_010,AUC Test,97,0.749532


In [214]:
auc_loss_scores_all_runs = pd.DataFrame(auc_loss_scores_all_runs)
auc_loss_scores_all_runs

,run,name,iteration,score
0,run_001,AUC Train,0,0.784185
1,run_001,AUC Train,1,0.796646
2,run_001,AUC Train,2,0.813906
3,run_001,AUC Train,3,0.831519
4,run_001,AUC Train,4,0.847915
...,...,...,...,...
3955,run_010,Cross-Entropy Loss,94,-0.158000
3956,run_010,Cross-Entropy Loss,95,-0.155441
3957,run_010,Cross-Entropy Loss,96,-0.159594
3958,run_010,Cross-Entropy Loss,97,-0.161458


In [215]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.set_title(f'Training curve on dataset {dataset_nr} {concept_changes_str}')
sns.lineplot(data=auc_scores_all_runs, x='iteration', y='score', hue='name')
ax.set_ylim(0.85,1) # 0.85, 1
ax.set_xlabel('Iteration')
ax.set_ylabel('AUC-ROC Score')

print(dataset_output_dir)
fig.savefig(f'{dataset_output_dir}/training_curve.png', bbox_inches='tight')
fig.clear()

c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v_randomfeat


<Figure size 800x800 with 0 Axes>

In [216]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title(f'Training curve on dataset {dataset_nr} {concept_changes_str}')
sns.lineplot(data=auc_loss_scores_all_runs, x='iteration', y='score', hue='name')
ax.set_ylim(top=1)
ax.set_xlabel('Iteration')
ax.set_ylabel('AUC-ROC/log10(Loss)')

fig.savefig(f'{dataset_output_dir}/training_curve_with_loss.png', bbox_inches='tight')
fig.clear()

<Figure size 800x800 with 0 Axes>

# Similarity between top scoring symptom-drug pairs

In [217]:
drug_symptom_pairs_per_run = []

for index, pred_path in enumerate(pred_folders_paths):
    with open(f'{pred_path}/candidates_per_symptom_{dataset_nr}_{embedding_method}{suffix}.pkl', 'rb') as f:
        loaded_list = pickle.load(f)
        
        drug_symptom_pairs = []
        
        for _, row in loaded_list.iterrows():
            symptom_id = row['Symptom']
            candidates = row['Candidates']
            
            for candidate in candidates:
                drug_symptom_pairs.append(tuple([symptom_id, candidate]))
                
        total_drug_symptom_pairs = len(drug_symptom_pairs)
    
    drug_symptom_pairs_per_run.append(drug_symptom_pairs)

In [218]:
similarity_matrix = {}
ratios_non_diagonals = []

for index1, pairs1 in enumerate(drug_symptom_pairs_per_run):
    
    similarities = {}
    
    for index2, pairs2 in enumerate(drug_symptom_pairs_per_run):
        overlap = set([tuple(sorted(ele)) for ele in pairs1]) & set([tuple(sorted(ele)) for ele in pairs2])
        ratio_overlap = len(overlap) / total_drug_symptom_pairs * 100
        
        similarities[f'run {index2+1}'] = ratio_overlap
        
        if index1 != index2:
            ratios_non_diagonals.append(ratio_overlap)
        
    similarity_matrix[f'run {index1+1}'] = similarities
    
similarity_matrix_df = pd.DataFrame(similarity_matrix)
similarity_matrix_df

,run 1,run 2,run 3,run 4,run 5,run 6,run 7,run 8,run 9,run 10
run 1,100.000000,3.703704,4.938272,2.469136,2.469136,1.234568,7.407407,1.234568,1.234568,0.000000
run 2,3.703704,100.000000,2.469136,1.234568,7.407407,4.938272,8.641975,6.172840,3.703704,3.703704
run 3,4.938272,2.469136,100.000000,2.469136,7.407407,2.469136,3.703704,2.469136,4.938272,2.469136
run 4,2.469136,1.234568,2.469136,100.000000,3.703704,2.469136,7.407407,1.234568,6.172840,4.938272
run 5,2.469136,7.407407,7.407407,3.703704,100.000000,7.407407,3.703704,1.234568,6.172840,6.172840
run 6,1.234568,4.938272,2.469136,2.469136,7.407407,100.000000,0.000000,4.938272,1.234568,2.469136
run 7,7.407407,8.641975,3.703704,7.407407,3.703704,0.000000,100.000000,3.703704,1.234568,6.172840
run 8,1.234568,6.172840,2.469136,1.234568,1.234568,4.938272,3.703704,100.000000,1.234568,2.469136
run 9,1.234568,3.703704,4.938272,6.172840,6.172840,1.234568,1.234568,1.234568,100.000000,2.469136
run 10,0.000000,3.703704,2.469136,4.938272,6.172840,2.469136,6.172840,2.469136,2.469136,100.000000


In [219]:
mean_overlap_ratio = np.mean(ratios_non_diagonals)
median_overlap_ratio = np.median(ratios_non_diagonals)
print('Mean:', mean_overlap_ratio)
print('Median:', median_overlap_ratio)

Mean: 3.676268861454046
Median: 3.7037037037037033


In [220]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title(f'Overlap ratio (Mean: {round(mean_overlap_ratio, 2)}, Median: {round(median_overlap_ratio, 2)}) between list of predicted symptom-drug pairs per run on dataset {dataset_nr} {concept_changes_str}')
sns.heatmap(similarity_matrix_df, annot=True, fmt='.1f', linewidths=0.5, ax=ax, cmap='RdYlGn')
ax.collections[0].set_clim(0,100)

fig.savefig(f'{dataset_output_dir}/overlap_between_runs.png', bbox_inches='tight')
fig.clear()

<Figure size 800x800 with 0 Axes>

In [221]:
for i in range(0, len(drug_symptom_pairs_per_run)):
    if i == 0:
        overlapping_pairs_all_runs = set(drug_symptom_pairs_per_run[i])
    else:
        overlapping_pairs_all_runs = overlapping_pairs_all_runs & set(drug_symptom_pairs_per_run[i])
            
print(f'There are {len(overlapping_pairs_all_runs)} symptom-drug pairs that are found in the top list of drug candidates in {len(drug_symptom_pairs_per_run)} runs: \n {overlapping_pairs_all_runs}')

with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_all_runs_{dataset_nr}_{embedding_method}{fixed_emb}.pkl', 'wb') as f:
    pickle.dump(overlapping_pairs_all_runs, f)

There are 0 symptom-drug pairs that are found in the top list of drug candidates in 10 runs: 
 set()


In [222]:
same_drug_symptom_pairs_thresholded = set()

threshold = 0.7
total_runs = len(drug_symptom_pairs_per_run)
min_nr_runs = int(threshold * total_runs)

for i in range(0, len(drug_symptom_pairs_per_run)):
    for pair in drug_symptom_pairs_per_run[i]:
        same_pairs = 0
        for j in range(0, len(drug_symptom_pairs_per_run)):
            for pair_to_compare in drug_symptom_pairs_per_run[j]:
                if pair == pair_to_compare:
                    same_pairs += 1
                        
        #print(f'For pair {pair} from run {i}, there are {same_pairs} same pairs found in list of all runs.')
        if same_pairs >= min_nr_runs:
            same_drug_symptom_pairs_thresholded.add(pair)
            
print(f'There are {len(same_drug_symptom_pairs_thresholded)} symptom-drug pairs that are found in the top list of drug candidates in at least {min_nr_runs} of the {total_runs} runs: \n {same_drug_symptom_pairs_thresholded}')

with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_{min_nr_runs}_runs_{dataset_nr}_{embedding_method}{fixed_emb}{suffix}.pkl', 'wb') as f:
    pickle.dump(same_drug_symptom_pairs_thresholded, f)

There are 0 symptom-drug pairs that are found in the top list of drug candidates in at least 7 of the 10 runs: 
 set()


# Symptoms related to DMD

In [223]:
edges = pd.read_csv(f'output/indexed_edges_{dataset_nr}{suffix}.csv')
edges

,head,label_head,class_head,index_head,relation,tail,label_tail,class_tail,index_tail,type
0,WormBase:WBGene00006787,unc-52,5,304,interacts with,WormBase:WBGene00003929,pat-2,5,1542,0
1,WormBase:WBGene00006787,unc-52,5,304,interacts with,WormBase:WBGene00006789,unc-54,5,6544,0
2,WormBase:WBGene00006787,unc-52,5,304,in orthology relationship with,ENSEMBL:ENSSSCG00000015555,LAMC1,5,9268,1
3,WormBase:WBGene00006787,unc-52,5,304,in orthology relationship with,ZFIN:ZDB-GENE-021226-3,lamc1,5,5387,1
4,WormBase:WBGene00006787,unc-52,5,304,in orthology relationship with,ENSEMBL:ENSOANG00000001050,ENSEMBL:ENSOANG00000001050,5,2204,1
...,...,...,...,...,...,...,...,...,...,...
85987,458,scopolamine butylbromide,4,5945,targets,P11229,Muscarinic acetylcholine receptor M1,6,5919,17
85988,OMIM:300377.0080,"DMD, IVS62, A-G, -285",11,1578,is allele of,HGNC:2928,DMD,5,3310,15
85989,5297,dacomitinib,4,8798,targets,P12931,Proto-oncogene tyrosine-protein kinase Src,6,2379,17
85990,ClinVarVariant:981988,NC_000023.11:g.(31875374_31929595)_(31968515_3...,11,8189,has affected feature,HGNC:2928,DMD,5,3310,11


In [224]:
if dataset_nr == 1:
    pheno_rel = 'has phenotype'
else:
    pheno_rel = 'associated with phenotype'
    
_, relation_labels = pd.factorize(edges['relation'])

disease_ID = 'MONDO:0010679'
relation_index = list(relation_labels).index(pheno_rel)

symptoms = edges[(edges['head'] == 'MONDO:0010679') & (edges['type'] == relation_index)]

print(f'A total of {symptoms.shape[0]} symptoms found that are associated with {disease_ID}')
symptoms

A total of 27 symptoms found that are associated with MONDO:0010679


,head,label_head,class_head,index_head,relation,tail,label_tail,class_tail,index_tail,type
41373,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0003707,Calf muscle pseudohypertrophy,9,5727,18
41374,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0003236,Elevated serum creatine kinase,9,1351,18
41375,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0001256,"Intellectual disability, mild",9,4533,18
41376,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0001265,Hyporeflexia,9,6515,18
41377,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0001290,Generalized hypotonia,9,5372,18
41378,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0003307,Hyperlordosis,9,2737,18
41379,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0003202,Skeletal muscle atrophy,9,788,18
41380,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0001263,Global developmental delay,9,7259,18
41381,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0002791,Hypoventilation,9,1682,18
41382,MONDO:0010679,Duchenne muscular dystrophy,3,1913,associated with phenotype,HP:0001371,Flexion contracture,9,5052,18


In [225]:
symptoms_df = symptoms[['tail', 'label_tail']]
symptoms_df = symptoms_df.rename(columns={'tail': 'Symptom ID', 'label_tail': 'Symptom'})
symptoms_df.to_csv(f'{curr_output_dir}/dmdrelatedsymptoms.csv', index=False)

# Check overlap between each model

In [226]:
nodes = pd.read_csv(f'output/indexed_nodes_{dataset_nr}.csv')
nodes.drop('index_id', axis=1, inplace=True)
nodes['semantic'] = nodes['semantic'].astype('category')
nodes

,id,semantic,label,semantic_id
0,MP:0004187,phenotype,cardia bifida,9
1,ZP:0100138,phenotype,muscle tendon junction myotome increased amoun...,9
2,MGI:1346525,gene,Sgcd,5
3,OMIM:300377.0044,variant,"DMD, LYS770TER",11
4,ZP:0002210,phenotype,posterior lateral line neuromast primordium mi...,9
...,...,...,...,...
10270,ZP:0014934,phenotype,atrioventricular valve development process qua...,9
10271,ENSEMBL:ENSCAFG00000011207,gene,ENSEMBL:ENSCAFG00000011207,5
10272,ENSEMBL:ENSXETG00000039922,gene,ENSEMBL:ENSXETG00000039922,5
10273,ENSEMBL:ENSACAG00000010058,gene,ENSEMBL:ENSACAG00000010058,5


In [227]:
if dataset_nr == 1:
    drug_semantic = 'DRUG'
else:
    drug_semantic = 'drug'

nodes[nodes['semantic'] == drug_semantic]

,id,semantic,label,semantic_id
89,606,drug,chloropyramine,4
98,3294,drug,iloperidone,4
103,1982,drug,olanzapine,4
129,1214,drug,fluphenazine enanthate,4
246,5303,drug,revefenacin,4
...,...,...,...,...
10140,3659,drug,yohimbine,4
10150,1836,drug,montelukast,4
10174,1402,drug,hyoscyamine,4
10201,MESH:D000077185,drug,Resveratrol,4


In [228]:
all_settings = [
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 1},
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 2}
]

curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')

overlapping_pairs = []

for setting in all_settings:
    dataset_output_dir = os.path.join(curr_output_dir, f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}')

    if not os.path.exists(dataset_output_dir):
        print('First, run the edge2vec embedding script. Then, run this script.')
    else:
        print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {dataset_output_dir}')
        
        with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_all_runs_{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}.pkl', 'rb') as f:
            loaded_list = pickle.load(f)
            print(loaded_list)
            overlapping_pairs.append(loaded_list)
            
            pair_dict_list = []
            for pair in loaded_list:
                symptom_id, drug_id = pair
                symptom_name = nodes.loc[nodes['id'] == symptom_id]['label'].iloc[0]
                drug_name = nodes.loc[nodes['id'] == drug_id]['label'].iloc[0]
                pair_dict_list.append({'Drug': drug_name, 'Symptom ID': symptom_id, 'Symptom': symptom_name})
                
            overlapping_all_runs_df = pd.DataFrame(pair_dict_list)
            overlapping_all_runs_df.to_csv(f'{dataset_output_dir}/symptom_drug_pairs_all_runs_{setting["dataset_nr"]}_{setting["embedding_method"]}.csv', index=False)

Output folder for dataset 2 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v
{('HP:0011675', '1576'), ('HP:0001635', '231'), ('HP:0002650', '5345'), ('HP:0003115', '231')}
Output folder for dataset 2 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v
{('HP:0001644', '1576'), ('HP:0001638', '1576'), ('HP:0001290', '269'), ('HP:0003115', '1576'), ('HP:0003236', '1576'), ('HP:0011675', '1576')}


In [229]:
dataset1_emb_overlap = overlapping_pairs[0].intersection(overlapping_pairs[1])
for pair in dataset1_emb_overlap:
    symptom_id, drug_id = pair
    symptom_name = nodes.loc[nodes['id'] == symptom_id]['label'].iloc[0]
    drug_name = nodes.loc[nodes['id'] == drug_id]['label'].iloc[0]
    print(drug_name, 'treats', symptom_name)

levosimendan treats Arrhythmia
